In [143]:
import pickle
import numpy as np
import pandas as pd
from geographiclib.geodesic import Geodesic
import os

In [144]:
def get_all_filenames(folder_path):
    filenames = []
    for filename in os.listdir(folder_path):
        filenames.append(filename)
    return filenames

In [145]:
folder_path = "D:\Spatial_interpolation\SSIN\data\india"
all_filenames = get_all_filenames(folder_path)

In [146]:
X_raw=np.load('data/all.npy')
X_raw.shape

(8784, 18, 103)

In [147]:
file_result=[]
for filename in all_filenames:
    x=pd.read_csv(folder_path+"/"+filename)
    x=np.array(x)
    x=np.reshape(x,(105120, 4 ,1))
    file_result.append(x)

In [148]:
result=file_result[0]

In [149]:
for i in range(1,len(file_result)):
    result = np.append(result,file_result[i] , axis=2)

In [150]:
result.shape

(105120, 4, 8)

In [151]:
X_raw=result

### X_train

In [152]:
j=3
train_data=X_raw[:,j,:]

In [153]:
columns_with_missing_data = np.any(np.isnan(train_data), axis=0)
missing_columns = np.where(columns_with_missing_data)[0]
result = np.delete(train_data, missing_columns, axis=1)
result = (result - np.min(result)) / (np.max(result) - np.min(result))

In [154]:
result.shape

(105120, 8)

In [155]:
train_list=[0,1,2,3]
test_list=[4,5,6,7]

In [156]:
train_data=result[:,train_list]
test_data=result[:,test_list]

In [ ]:
train_data = np.transpose(train_data, (0, 2, 1))
test_data = np.transpose(test_data, (0, 2, 1))

In [157]:
# train_data = np.reshape(train_data, (105120, 4,1))
# test_data = np.reshape(test_data, (105120, 4,1))

In [158]:
delet_list=[]
for i in range(len(train_data)):
    temp=np.reshape(train_data[i],(4))
    if (len(set(temp)) == 1):
        delet_list.append(i)

In [159]:
train_data=np.delete(train_data,delet_list,axis=0)

## timestaps

In [160]:
years=['2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','2023']
months=['01','02','03','04','05','06','07','08','09','10','11','12']
days=['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31']
hours=['00','01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23']


In [161]:
timestaps=[]
for year in years:
    for month in  months:
        for day in days:
            for hour in hours:
                x=str(year)+str(month)+str(day)+"-"+str(hour)
                timestaps.append(x)

In [162]:
timestaps_result=timestaps[:105120]

In [163]:
timestaps_train_result=timestaps[:len(train_data)]

In [164]:
timestap_result=np.array(timestaps_result)

In [165]:
timestaps_train_result=np.array(timestaps_train_result)

In [166]:
timestap_result.shape

(105120,)

In [167]:
timestaps_train_result.shape

(104823,)

### invid_mask掩码

In [168]:
def generate1_random_array(length):
    array = []
    for _ in range(length):
        array.append(False)
    return np.array(array)
 
# 生成长度为10的随机数组
train_invid_mask=[]
test_invid_mask=[]
for i in range(len(timestap_result)):
    # random_array = generate1_random_array(len(train_data[0]))
    random_array1 = generate1_random_array(len(test_data[0]))
    # train_invid_mask.append(random_array)
    test_invid_mask.append(random_array1)

for i in range(len(timestaps_train_result)):
    random_array = generate1_random_array(len(train_data[0]))
    # random_array1 = generate1_random_array(len(test_data[0]))
    train_invid_mask.append(random_array)
    # test_invid_mask.append(random_array1)
    
train_invid_mask=np.array(train_invid_mask)
test_invid_mask=np.array(test_invid_mask)


### test_mask掩码

In [169]:
import random
 
def generate_random_array(length):
    array = []
    for _ in range(length):
            array.append(random.choice([False,True]))
    return np.array(array)

# 生成长度为10的随机数组
test_mask=[]
random_array=generate_random_array(test_data.shape[1])
for i in range(len(timestap)):
    test_mask.append(random_array)
test_mask=np.array(test_mask)

### 地理位置

#### 创建r_pos_mat

In [170]:
stations = pd.read_csv("D:\Spatial_interpolation\SSIN\data\Locations_of_India.csv")

In [171]:
df_train=stations.iloc[0:4]
df_test=stations.iloc[4:8]

In [172]:
def calc_dist_angle_mat(df, out_path):
    lons, lats = df["lon"].values, df["lat"].values
    dist_angle_mat = np.zeros((len(lons), len(lons), 2))

    for i in range(len(lons)):
        for j in range(len(lons)):
            dist = Geodesic.WGS84.Inverse(lats[i], lons[i], lats[j], lons[j])
            dist_angle_mat[i, j, 0] = dist["s12"] / 1000.0  # distance, km
            dist_angle_mat[i, j, 1] = dist["azi1"]  # azimuth at the first point in degrees

    print(dist_angle_mat.shape)
    # print(dist_angle_mat)
    np.save(out_path, dist_angle_mat)

In [173]:
if __name__ == "__main__":
    base_dir = "../data"

    # HK dataset
    # info_path = f"{base_dir}/HK_123_data/hko_stations_info.csv"
    # out_dir = f"{base_dir}"

    # BW dataset
    # info_path = f"{base_dir}/BW_132_data/BW_stations_info.csv"
    out_dir = f"{base_dir}"

    # os.makedirs(out_dir, exist_ok=True)

    out_name = "dist_angle_mat_train.npy"
    out_path = f"{out_dir}/{out_name}"
    calc_dist_angle_mat(df_train, out_path)

(4, 4, 2)


In [174]:
if __name__ == "__main__":
    base_dir = "../data"

    # HK dataset
    # info_path = f"{base_dir}/HK_123_data/hko_stations_info.csv"
    out_dir = f"{base_dir}"

    # BW dataset
    # info_path = f"{base_dir}/BW_132_data/BW_stations_info.csv"
    # out_dir = f"{base_dir}/BW_132_data"

    # os.makedirs(out_dir, exist_ok=True)

    out_name = "dist_angle_mat_test.npy"
    out_path = f"{out_dir}/{out_name}"
    calc_dist_angle_mat(df_test, out_path)

(4, 4, 2)


In [175]:
def generate_stat_from_train_data(train_info_df, r_pos_mat):
    # info_df = pd.read_csv(info_path)
    # ori_r_pos_mat = np.load(relative_pos_mat_path)

    # is_test = info_df["is_test"].values
    # train_mask = np.where(is_test == 0, True, False)
    # train_info_df = info_df.loc[train_mask, :]

    lat_mean, lat_std, lat_max, lat_min = train_info_df["lat"].mean(), train_info_df["lat"].std(ddof=0), \
                                          train_info_df["lat"].max(), train_info_df["lat"].min()
    lon_mean, lon_std, lon_max, lon_min = train_info_df["lon"].mean(), train_info_df["lon"].std(ddof=0), \
                                          train_info_df["lon"].max(), train_info_df["lon"].min()

    # indexes = np.where(train_mask)[0]
    # idx_i, idx_j = np.ix_(indexes, indexes)
    # r_pos_mat = ori_r_pos_mat[idx_i, idx_j, :]

    r_dist_mat = r_pos_mat[:, :, 0]
    r_angle_mat = r_pos_mat[:, :, 1]

    r_dist_mean, r_dist_std, r_dist_max, r_dist_min = np.mean(r_dist_mat), np.std(r_dist_mat), \
                                                      np.max(r_dist_mat), np.min(r_dist_mat),
    r_angle_mean, r_angle_std, r_angle_max, r_angle_min = np.mean(r_angle_mat), np.std(r_angle_mat), \
                                                          np.max(r_angle_mat), np.min(r_angle_mat)

    stat_dict = {}
    stat_dict["lat_mean"], stat_dict["lat_std"], stat_dict["lat_max"], stat_dict["lat_min"] = \
        lat_mean, lat_std, lat_max, lat_min
    stat_dict["lon_mean"], stat_dict["lon_std"], stat_dict["lon_max"], stat_dict["lon_min"] = \
        lon_mean, lon_std, lon_max, lon_min

    stat_dict["r_dist_mean"], stat_dict["r_dist_std"], stat_dict["r_dist_max"], stat_dict["r_dist_min"] = \
        r_dist_mean, r_dist_std, r_dist_max, r_dist_min
    stat_dict["r_angle_mean"], stat_dict["r_angle_std"], stat_dict["r_angle_max"], stat_dict["r_angle_min"] = \
        r_angle_mean, r_angle_std, r_angle_max, r_angle_min

    print("Calculates the statistics of training data. Done!")

    # with open("./data/hk_data_stats.pkl".format(out_name), "wb") as fp:
    #     pickle.dump(stat_dict, fp)

    return stat_dict

In [176]:
train_df=np.load("D:\Spatial_interpolation\data\dist_angle_mat_train.npy")
test_df=np.load("D:\Spatial_interpolation\data\dist_angle_mat_test.npy")

In [177]:
stat_dict=generate_stat_from_train_data(df_train, train_df)

Calculates the statistics of training data. Done!


In [178]:
train_df[:, :, 0] = (train_df[:, :, 0] - stat_dict["r_dist_mean"]) / stat_dict["r_dist_std"]
train_df[:, :, 1] = (train_df[:, :, 1] - stat_dict["r_angle_mean"]) / stat_dict["r_angle_std"]

In [179]:
test_df[:, :, 0] = (test_df[:, :, 0] - stat_dict["r_dist_mean"]) / stat_dict["r_dist_std"]
test_df[:, :, 1] = (test_df[:, :, 1] - stat_dict["r_angle_mean"]) / stat_dict["r_angle_std"]

In [180]:
test={"test_data":test_data,"invalid_masks":test_invid_mask,"test_masks":test_mask,"r_pos_mat":test_df,"timestamps":timestap_result}

In [181]:
train={"train_data":train_data,"invalid_masks":train_invid_mask,"r_pos_mat":train_df,"timestamps":timestaps_train_result}

In [141]:
test["timestamps"].shape

(105120,)

In [182]:
with open('train.pkl', 'wb') as f:
    pickle.dump(train, f)


with open('test.pkl', 'wb') as f:
    pickle.dump(test, f)